# Ejemplos TCP + UDP

---

# 1. Ejemplo TCP : Pedir mis datos al servidor

## 1.1 Server

In [ ]:

import java.io.DataInputStream;
import java.io.DataOutputStream;
import java.io.IOException;
import java.net.ServerSocket;
import java.net.Socket;

public class Server {
	
	
	/**
	 * Puerto por donde el servidor atendera a los clientes
	 */
	public static final int PORT = 8000;
	/**
	 * El servidor dispone de un serversocket, para permitir la conexion a los clientes
	 */
	private static  ServerSocket serverSocket;
	/**
	 * El servidor dispone de un socket para atender a cada cliente por individual
	 */
	private static Socket socket;
	

	
	
	public static void main(String[] args) {
		
		DataInputStream in;
		DataOutputStream out;
		
		try {
			serverSocket = new ServerSocket(PORT);
			System.out.println("::Servidor escuchando a los posibles clientes::");
			
			while(true) {
				
				socket = serverSocket.accept();
				System.out.println("El cliente se ha conectado!");
				in = new DataInputStream(socket.getInputStream());
				out = new DataOutputStream(socket.getOutputStream());
				String mensajeObtenidoCliente = in.readUTF();
				if(mensajeObtenidoCliente.compareTo("LISTO")==0) {
					String respuestaServer = socket.getInetAddress().getHostAddress()+";"+socket.getPort();
					out.writeUTF(respuestaServer);
					
				}
				socket.close();
				System.out.println("::El cliente fue desconectado del server::");
			
			
			}
			
			
			
			
			
			
			
		} catch (IOException e) {
			// TODO Auto-generated catch block
			e.printStackTrace();
		}
		
		
		
		
	}



	
	

}


## 1.2 Cliente

In [ ]:

import java.io.BufferedReader;
import java.io.BufferedWriter;
import java.io.DataInputStream;
import java.io.DataOutputStream;
import java.io.InputStreamReader;
import java.io.OutputStreamWriter;
import java.net.Socket;

public class Client {
	
	/*
	 * 
	 * Direccion local de la maquina
	 */
	public static final String LOCAL_HOST = "localhost";
	/**
	 * Puerto por donde se establecera la conexion
	 */
	public static final int PORT = 8000;
	/**
	 * Socket que permitira la conexion con el servidor
	 */
	private static Socket socket;
	
	/**
	 * Main
	 * @param args
	 */
	public static void main(String[] args) {
		
		DataInputStream in;
		DataOutputStream out;

		try {
			
			BufferedReader br = new BufferedReader(new InputStreamReader( System.in));
			BufferedWriter bw = new BufferedWriter(new OutputStreamWriter(System.out));
			
			System.out.println("::Cliente disponible para ser atendido:: \nIngrese "
					+ "La palabra LISTO para obtener su dir ip y puerto de conexion!!::");
			
			socket = new Socket(LOCAL_HOST, PORT);
			String mensaje = br.readLine();
			in = new DataInputStream(socket.getInputStream());
			out = new DataOutputStream(socket.getOutputStream());
			out.writeUTF(mensaje);
			String mensajeDelServidor = in.readUTF();
			String[] mensajeServ = mensajeDelServidor.split(";");
			bw.write("Su Direccion ip es : " + mensajeServ[0]+"\n");
			bw.write("Su Puerto de conexion es : " + mensajeServ[1]);
			bw.flush();
			bw.close();
			br.close();
			socket.close();
			in.close();
			out.close();

		} catch (Exception e) {
			// TODO: handle exception
		}
		
		
	}
	

}


Link : 

## 1.3 Reto TCP

Link : https://github.com/JulioCe98/Networks_Monitory/tree/master/Retos

---

# 2 UDP

## 2.1 Datagram Channel

### 2.1.1 Server

In [ ]:
 package Schema;


import java.net.DatagramSocket;
import java.net.InetSocketAddress;
import java.net.SocketAddress;
import java.nio.ByteBuffer;
import java.nio.channels.DatagramChannel;

public class UDPEchoServerWithChannels {
	
	/**
	 * 
	 * Introduccion
	 * 
	 * DatagramChannel : 
	 * 
	 * 1) Es mucho mas "No bloqueante" que UDP
	 * 2) Si mucho tiene que esperar a que se borre el buffer local
	 * 3) No debe esperar a que un cliente este listo para recibir datos
	 * 4) No se detiene si un cliente esta esperando a que otro reciba
	 * 
	 * 
	 * 
	 * 
	 * ECHO 
	 * 
	 * 1) Permite visualizar si se estan perdiendo paquetes al enviar 100 enteros
	 * 
	 */
	
	
	
	
	public static final int PORT = 8500;
	public static final int MAX_PACKET_SIZE = 65507;

	public static void main(String[] args) {
		try {
			DatagramChannel channel = DatagramChannel.open();
			DatagramSocket socket = channel.socket();
			SocketAddress address = new InetSocketAddress(PORT);
			socket.bind(address);
			ByteBuffer buffer = ByteBuffer.allocateDirect(MAX_PACKET_SIZE);
			while(true) {
				SocketAddress client = channel.receive(buffer);
				//resetea el buffer para recibir nuevos datos (vuelve a la posicion 0)
				buffer.flip();
				channel.send(buffer, client);
				//limpia el buffer
				buffer.clear();
			}
		}
		catch (Exception e) {
			System.err.println(e);
			}
		
		
		
		
		
	}




}


### 2.1.2 Client

In [ ]:
package Schema;

import java.net.InetSocketAddress;
import java.net.SocketAddress;
import java.nio.ByteBuffer;
import java.nio.channels.DatagramChannel;
import java.nio.channels.SelectionKey;
import java.nio.channels.Selector;
import java.util.Iterator;
import java.util.Set;

public class UDPEchoClientWithChannels {

	public static final int PORT = 8500;
	public static final int LIMIT = 100;

	public static void main(String[] args) {
		SocketAddress remote = null;
		try {
			remote = new InetSocketAddress("127.0.0.1", PORT);
			
		} catch (RuntimeException e) {
			
			System.err.println("Usage: Java UDPEchoClientWithChannels host");

		}
		
		try (DatagramChannel channel = DatagramChannel.open()){
			/**
* protected abstract void implConfigureBlocking(boolean block)
                                       throws IOException
Adjusts this channel's blocking mode.
This method is invoked by the configureBlocking method in order to perform the actual work of changing the blocking mode. This method is only invoked if the new mode is different from the current mode.

Parameters:
block - If true then this channel will be placed in blocking mode; if false then it will be placed non-blocking mode
Throws:
IOException - If an I/O error occurs

Perform : Realizar;
			 */
			channel.configureBlocking(false);
			channel.connect(remote);
			Selector selector = Selector.open();
			/**
* public final SelectionKey register(Selector sel,
                                   int ops)
                            throws ClosedChannelException
Registers this channel with the given selector, returning a selection key.
An invocation of this convenience method of the form

sc.register(sel, ops)
behaves in exactly the same way as the invocation
sc.register(sel, ops, null)
Parameters:
sel - The selector with which this channel is to be registered
ops - The interest set for the resulting key
Returns:
A key representing the registration of this channel with the given selector
			 */
			channel.register(selector, SelectionKey.OP_READ | SelectionKey.OP_WRITE);
			/**
			 * public static ByteBuffer allocate(int capacity)
Allocates a new byte buffer.
The new buffer's position will be zero, its limit will be its capacity, its mark will be undefined, and each of its elements will be initialized to zero. 
It will have a backing array, and its array offset will be zero.

Parameters:
capacity - The new buffer's capacity, in bytes
Returns:
The new byte buffer
			 */
			ByteBuffer buffer = ByteBuffer.allocate(4);
			int n = 0;
			int numbersRead = 0;
			while(true) {
				if(numbersRead == LIMIT) break;
				//wait one minute for a connection
				selector.select(60000);
				Set<SelectionKey> readyKeys = selector.selectedKeys();
				if(readyKeys.isEmpty() && n==LIMIT) {
					/**
					 *  All packets have been written and it doesn't look like any 
					 *  more are will arrive from the network
					 */
					break;
				}
				else {
					Iterator<SelectionKey> iterator = readyKeys.iterator();
					while(iterator.hasNext()) {
						SelectionKey key = (SelectionKey)iterator.next();
						iterator.remove();
						if(key.isReadable()) {
							buffer.clear();
							channel.read(buffer);
							buffer.flip();
							int echo = buffer.getInt();
							System.out.println("Read : " + echo);
							numbersRead++;
						}
						if (key.isWritable()) {
							buffer.clear();
							buffer.putInt(n);
							buffer.flip();
							channel.write(buffer);
							System.out.println("Wrote : " + n);
							n++;
							if (n==LIMIT) {
								key.interestOps(SelectionKey.OP_READ);
							}
						}
					}
				}
			}
			System.out.println("Echoed " + numbersRead + " out of " + LIMIT + " sent");
			System.out.println("Success rate : " + (100.0 * (numbersRead/LIMIT)) + "%");
			
		} catch (Exception e) {
			System.err.println(e);
		}
			
		
	}
	
}


Link: 

## 2.1.3 Funcionamiento

<img src="./Resources/dc1.png">

---

## 3 UDP Scanner

In [ ]:
package Schema;

import java.net.DatagramSocket;
import java.net.SocketException;

public class ScannerUDP {
	
	
	public static void Scanning() {
		for (int i = 1024; i <= 65535; i++) {
			try {
				DatagramSocket dt = new DatagramSocket(i);
				dt.close();
			} catch (SocketException e) {
				System.out.println("There is a server on port : "+ i );
			}
		}
	}
	
	public static void main(String[] args) {
		
		Scanning();
		
	}

}


### 3.1 Funcionamiento

1. **Dar click en ejecutar**

<img src="./Resources/sc1.png">

Como se puede evidenciar en la anterior foto, se puede ver que el algoritmo realiza un **PING** a los puertos para saber que servicio tiene. Una vez con este servicio, verifica si es **UDP** y lo imprime por consola!

---

# 4. Reto 5 :Ejemplos propuestos por los estudiantes

Link: https://github.com/JulioCe98/Networks_Monitory/tree/master/Retos